In [31]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urls
import re

In [26]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
nltk.download('cmudict')
from nltk.corpus import cmudict
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\farzi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\farzi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\cmudict.zip.


In [47]:
df = pd.read_excel('20211030 Test Assignment\Output Data Structure.xlsx')
l1= list(inputlinks.URL)

In [48]:
df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Function to count syllables in a word
def count_syllables(word):
    word_stripped = word.rstrip('es').rstrip('ed')
    if word.lower() in pronouncing_dict:
        return max([len(list(y for y in x if y[-1].isdigit())) for x in pronouncing_dict[word.lower()]])
    else:
        return 0



In [45]:
def feature_extraction(url):
    
    #  Create an empty set to store the lowercase stop words
    all_stop_words = set()

    # List of stop words files
    stop_words_files = [urls.sw1, urls.sw2, urls.sw3, urls.sw4, urls.sw5, urls.sw6]
    # Load the list of positive words from a file
    with open(urls.pw, 'r', encoding='iso-8859-1') as positive_words_file:
        positive_words = set(positive_words_file.read().splitlines())
    with open(urls.nw, 'r', encoding='iso-8859-1') as negative_words_file:
        negative_words = set(negative_words_file.read().splitlines())

    # Iterate through each stop words file and add lowercase stop words to the set
    for stop_words_file_name in stop_words_files:
        with open(stop_words_file_name, 'r', encoding='iso-8859-1') as stop_words_file:
            lowercase_stop_words = set(word.lower() for word in stop_words_file.read().splitlines())
            all_stop_words.update(lowercase_stop_words)

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Identify and remove header and footer elements
        header = soup.find('header')  # Replace with the correct tag for header
        if header:
            header.extract()
        
        footer = soup.find('footer')  # Replace with the correct tag for footer
        if footer:
            footer.extract()
        
        # Extract the remaining text
        all_text = soup.get_text()
        
        # Remove blank lines from the text
        lines = all_text.split('\n')
        non_blank_lines = [line for line in lines if line.strip() != '']
        
        # Join the non-blank lines back together
        cleaned_text = '\n'.join(non_blank_lines)
        
        # Exclude the first 10 characters, first 160 lines, and the last 40 lines
        characters_to_exclude_start = 21
        lines_to_exclude_start = 162
        lines_to_exclude_end = 42
        
        cleaned_text_lines = cleaned_text.split('\n')[lines_to_exclude_start:-lines_to_exclude_end]
        cleaned_text_lines[0] = cleaned_text_lines[0][characters_to_exclude_start:]
        
        final_text = '\n'.join(cleaned_text_lines)
        

        
        # Load the CMU Pronouncing Dictionary for syllable counting
        pronouncing_dict = cmudict.dict()
        # Tokenize the text into sentences
        sentences = sent_tokenize(final_text)
        # Count the number of sentences
        num_sentences = len(sentences)
        
        # stop words not removed
        # Tokenize the words in the extracted text
        all_words = final_text.split()
        avg_sent_len = len(all_words)/num_sentences
        complex_words = [word for word in all_words if count_syllables(word) > 2]
        percent_of_complex_words = len(complex_words)/len(all_words)
        Fog_Index = 0.4 * (avg_sent_len + percent_of_complex_words)
        # Avg_word_count_per_sentence = len(words)/num_sentences

        #remove stopwords
        pronouns = ['I', 'we', 'my','ours', 'We', 'My', 'Ours']
        punct_cleaned_txt=re.sub(r'[?!,.]', '', final_text)
        words = punct_cleaned_txt.split()
        personal_pronoun_count = sum(1 for word in words if word in personal_pronouns)
        # Filter out stop words (case-insensitive)
        filtered_words = [word for word in words if word.lower() not in all_stop_words]
        avg_syll_count_per_word = sum(count_syllables(word) for word in filtered_words)/len(filtered_words)
        avg_word_length = sum(len(word) for word in filtered_words)/len(filtered_words)
        final_cleaned_text = ' '.join(filtered_words)
        
        
        word_count = len(words)
        positive_word_count = sum(1 for word in filtered_words if word.lower() in positive_words)
        negative_word_count = sum(1 for word in filtered_words if word.lower() in negative_words)
        Polarity_Score  =  (positive_word_count- negative_word_count)/  ((positive_word_count+negative_word_count)+ 0.000001)
        Subjectivity_Score  =  (positive_word_count + negative_word_count)/ (word_count + 0.000001)

        returndict = {'POSITIVE SCORE':positive_word_count,'NEGATIVE SCORE':negative_word_count,'POLARITY SCORE':Polarity_Score,'SUBJECTIVITY SCORE':Subjectivity_Score,'AVG SENTENCE LENGTH':avg_sent_len,'PERCENTAGE OF COMPLEX WORDS':percent_of_complex_words,'FOG INDEX':Fog_Index,'AVG NUMBER OF WORDS PER SENTENCE':avg_sent_len,'COMPLEX WORD COUNT':len(complex_words),'WORD COUNT':word_count,'SYLLABLE PER WORD':avg_syll_count_per_word,'PERSONAL PRONOUNS':personal_pronoun_count,'AVERAGE WORD LENGTH':avg_word_length}
        # print(positive_word_count, negative_word_count,Polarity_Score,Subjectivity_Score,avg_sent_len,percent_of_complex_words,Fog_Index,personal_pronoun_count,avg_word_length)
        return returndict

    else:
        print (f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return

        

In [50]:
# Iterate through the links column and populate feature columns
for index, row in df.iterrows():
    link = row['URL']
    features = feature_extraction(link)
    
    if features is None:
        # Handle the case where features are None
        print(f"Features not extracted for link at index {index}: {link}")
        continue
    
    # Update corresponding feature columns with extracted values
    for feature_column in features.keys():
        df.at[index, feature_column] = features[feature_column]


Failed to retrieve the webpage. Status code: 404
Features not extracted for link at index 24: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/
Failed to retrieve the webpage. Status code: 404
Features not extracted for link at index 37: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/


In [51]:
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,AVERAGE WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,82.0,23.0,0.561905,0.061692,21.275000,0.173325,8.579330,21.275000,295.0,1702.0,2.338710,2.0,NaN,7.492944
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,41.0,13.0,0.518519,0.082695,26.120000,0.182236,10.520894,26.120000,119.0,653.0,2.325779,3.0,NaN,7.688385
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,23.0,27.0,-0.080000,0.041841,17.333333,0.155518,6.995541,17.333333,186.0,1195.0,1.917282,3.0,NaN,7.220089
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,42.0,26.0,0.235294,0.053501,21.183333,0.178600,8.544773,21.183333,227.0,1271.0,2.040260,6.0,NaN,7.137662
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,42.0,26.0,0.235294,0.053501,21.183333,0.178600,8.544773,21.183333,227.0,1271.0,2.040260,6.0,NaN,7.137662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,8.0,28.0,-0.555556,0.050633,23.700000,0.151899,9.540759,23.700000,108.0,711.0,1.827930,1.0,NaN,6.842893
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,24.0,58.0,-0.414634,0.048009,34.160000,0.132904,13.717162,34.160000,227.0,1708.0,1.716037,2.0,NaN,6.720123
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,93.0,30.0,0.512195,0.069492,25.285714,0.160452,10.178467,25.285714,284.0,1770.0,1.967181,0.0,NaN,6.916023
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,32.0,22.0,0.185185,0.039216,23.338983,0.139434,9.391367,23.338983,192.0,1377.0,1.915119,5.0,NaN,7.104775


In [52]:
df = df.drop(columns=['AVG WORD LENGTH'])

In [53]:
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVERAGE WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,82.0,23.0,0.561905,0.061692,21.275000,0.173325,8.579330,21.275000,295.0,1702.0,2.338710,2.0,7.492944
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,41.0,13.0,0.518519,0.082695,26.120000,0.182236,10.520894,26.120000,119.0,653.0,2.325779,3.0,7.688385
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,23.0,27.0,-0.080000,0.041841,17.333333,0.155518,6.995541,17.333333,186.0,1195.0,1.917282,3.0,7.220089
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,42.0,26.0,0.235294,0.053501,21.183333,0.178600,8.544773,21.183333,227.0,1271.0,2.040260,6.0,7.137662
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,42.0,26.0,0.235294,0.053501,21.183333,0.178600,8.544773,21.183333,227.0,1271.0,2.040260,6.0,7.137662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,8.0,28.0,-0.555556,0.050633,23.700000,0.151899,9.540759,23.700000,108.0,711.0,1.827930,1.0,6.842893
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,24.0,58.0,-0.414634,0.048009,34.160000,0.132904,13.717162,34.160000,227.0,1708.0,1.716037,2.0,6.720123
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,93.0,30.0,0.512195,0.069492,25.285714,0.160452,10.178467,25.285714,284.0,1770.0,1.967181,0.0,6.916023
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,32.0,22.0,0.185185,0.039216,23.338983,0.139434,9.391367,23.338983,192.0,1377.0,1.915119,5.0,7.104775


In [55]:
# Define the Excel file path
excel_file_path = '20211030 Test Assignment\Output Data Structure.xlsx'

# Write the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print("Data written to Excel file:", excel_file_path)

Data written to Excel file: 20211030 Test Assignment\Output Data Structure.xlsx
